In [2]:
%pip install openCV-python
# %pip install python
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import numpy as np
import maths
from cvzone.HandTrackingModule import HandDetector
import time
from cvzone.ClassificationModule import Classifier
# cap = cv2.VideoCapture
cap = cv2.VideoCapture(0)

detector= HandDetector(maxHands=1)
classifier = Classifier("Model/keras_model.h5","Model/labels.txt")
# this keras model is the  name of downloaded file from online when we compile the given data of A B C pic folder.

offset = 20 

imgSize= 300

folder= "Data/A"
# folder= "Data/B"
# folder= "Data/C"

counter=0

labels= ["A", "B", "C"]

while True:
    success, img = cap.read()
    imgOutput= img.copy()
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]

        x,y,w,h= hand['bbox']
        
        imgWhite= np.ones((imgSize, imgSize, 3), np.uint8)*255

        imgCrop= img[y- offset:y+h+offset, x-offset:x+w+offset]
        
        imgCropShape=imgCrop.shape
        
        aspectRAtio=h/w
        if aspectRatio >1:
            k=imgSize/h
            wCal= math.ceil(k*w)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            imgResizeShape = imgResize.shape
            wGap=math.ceil((imgSize- wCal)/2)
            imgWhite[:, wGap:wCal+wGap] = imgResize
            prediction, index= classifier.getPrediction(imgWhite , draw= False)
            print(prediction , index)


        else:
            k=imgSize/w
            wCal= math.ceil(k*h)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            imgResizeShape = imgResize.shape
            hGap=math.ceil((imgSize- hCal)/2)
            imgWhite[ hGap:hCal+hGap, :] = imgResize
            prediction, index= classifier.getPrediction(imgWhite)
        
        cv2.rectangle(imgOutput,(x-offset,y-offset),(x-offset+150,y-offset+50),(255,0,255),cv2.FILLED)

        cv2.putText(imgOutput,labels[index], (x,y-20),cv2.FONT_HERSHEY_COMPLEX,2,(255,0,255),2)

        cv2.rectangle(imgOutput,(x-offset,y-offset),(x+w+offset,y+h+offset),(255,0,255),4)

        
        cv2.imshow("imagecrop", imgCrop)
        
        cv2.imshow("imageWhite", imgWhite)
    
    cv2.imshow("image",imgOutput)
    
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
         break